In [ ]:
import torch
import torchaudio
import torchaudio.transforms as T
#from torch.nn import functional as F
from torch import flatten
from platform import python_version
sample_rate = 16000
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print('Python Version :',python_version())
print('Torch Version:',torch.__version__)
print('Torch Audio Version :',torchaudio.__version__)
print('Is CUDA available? :', torch.cuda.is_available())
print('Device name :',torch.cuda.get_device_name(torch.cuda.current_device()))
print('Backend de audio disponibles',str(torchaudio.list_audio_backends()))

In [ ]:
import os   
def create_dataset(dataset_path):
    clases = {'fake':torch.tensor([0]), 'real':torch.tensor([1])}
    X=torch.tensor(())
    y=torch.tensor(())
    class_path = os.listdir(dataset_path)
    for clase in class_path:
        file_path = os.path.join(dataset_path,clase)
        for file in os.listdir(file_path):
            audio,audio_rate = torchaudio.load(os.path.join(file_path,file), backend='soundfile')
            #mfcc = transform(audio)
            X = torch.cat((X,audio),0)
            y = torch.cat((y,clases[clase]),0)
    #return sub_dt(X,y)
    return X.to(device), y.to(device)

In [ ]:
X_testing, y_testing = create_dataset('../for-2seconds/testing')#Manually created from training set
X_training, y_training = create_dataset('../for-2seconds/training')
X_valid, y_valid = create_dataset('../for-2seconds/validation')


In [ ]:
X_training.shape, y_training.shape, X_valid.shape, y_valid.shape, X_testing.shape, y_testing.shape

In [ ]:
import matplotlib.pyplot as plt

n_fft = 2048
win_length = None
hop_length = 512
n_mels = 256
n_mfcc = 8
mfcc_transform = T.MFCC(
    sample_rate=16000,
    n_mfcc=n_mfcc,
    melkwargs={"n_fft": n_fft,"n_mels": n_mels,"hop_length": hop_length,"mel_scale": "htk",}
)

n_fft = 2048
win_length = None
hop_length = 512
n_lfcc = 8

lfcc_transform = T.LFCC(
    sample_rate=16000,
    n_lfcc=n_lfcc,
    speckwargs={
        "n_fft": n_fft,
        "win_length": win_length,
        "hop_length": hop_length,
    },
)

def plot_waveform(waveform, sr, title="Waveform", ax=None):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sr

    if ax is None:
        _, ax = plt.subplots(num_channels, 1)
    ax.plot(time_axis, waveform[0], linewidth=1)
    ax.grid(True)
    ax.set_xlim([0, time_axis[-1]])
    ax.set_title(title)

def plot_spectrogram(spectrogram, title=None, ylabel="freq_bin", ax=None, ):
    if ax is None:
        _, ax = plt.subplots(1, 1)
    if title is not None:
        ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.imshow(spectrogram, origin="lower", aspect="auto", interpolation="nearest")

In [ ]:
audio,audio_rate = torchaudio.load('../for-2seconds/validation\\fake\\file15502.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav', backend='soundfile')
#plot_waveform(audio)
fig, axs = plt.subplots(3, 1)
plot_waveform(audio, audio_rate, title="Original waveform", ax=axs[0])
plot_spectrogram(mfcc_transform(audio)[0], title="MFCC", ax=axs[1])
plot_spectrogram(lfcc_transform(audio)[0], title="LFCC", ax=axs[2])
fig.tight_layout()

In [ ]:
audio,audio_rate = torchaudio.load('../for-2seconds/validation\\real\\file5.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav', backend='soundfile')
#plot_waveform(audio)
fig, axs = plt.subplots(3, 1)
plot_waveform(audio, audio_rate, title="Original waveform", ax=axs[0])
plot_spectrogram(mfcc_transform(audio)[0], title="MFCC", ax=axs[1])
plot_spectrogram(lfcc_transform(audio)[0], title="LFCC", ax=axs[2])
fig.tight_layout()

In [ ]:
audio.shape

In [ ]:
mfcc = mfcc_transform(audio)
mfcc = mfcc[:, :]
mfcc.shape,flatten(mfcc,1).shape

In [ ]:
lfcc = lfcc_transform(audio)
lfcc = lfcc[:, :]
lfcc.shape,flatten(lfcc,1).shape

In [ ]:
from timeit import default_timer as timer 
from tqdm.auto import tqdm
import numpy as np
import matplotlib.ticker as mticker
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
import torch.nn as nn

s = nn.Softmax(dim=0)
def plot_loss_acc(hist):
    x_arr=np.arange(len(hist[0]))+1
    fig=plt.figure(figsize=(12,4))
    ax=fig.add_subplot(1,2,1)
    ax.ticklabel_format(style='plain', axis='x', useOffset=False)
    ax.xaxis.set_major_locator(mticker.MultipleLocator(1))
    ax.plot(x_arr,hist[0], '-o', label='Train Loss')
    ax.plot(x_arr,hist[1], '--<', label='Valid Loss')
    ax.legend(fontsize=15)
    ax.set_xlabel('Epoch', size=15)
    ax.set_ylabel('Loss', size=15)
    ax=fig.add_subplot(1,2,2)
    ax.ticklabel_format(style='plain', axis='x', useOffset=False)
    ax.xaxis.set_major_locator(mticker.MultipleLocator(1))
    ax.plot(x_arr,hist[2], '-o', label='Train Acc')
    ax.plot(x_arr,hist[3], '--<', label='Valid Acc')
    ax.legend(fontsize=15)
    ax.set_xlabel('Epoch', size=15)
    ax.set_ylabel('Accuracy', size=15)
    ax.set_ylim(0.45,1.05)
    plt.show()
    
def print_train_time(start: float, end: float, device: torch.device = None):
    """Prints difference between start and end time.

    Args:
        start (float): Start time of computation (preferred in timeit format). 
        end (float): End time of computation.
        device ([type], optional): Device that compute is running on. Defaults to None.

    Returns:
        float: time between start and end in seconds (higher is longer).
    """
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time
#torch.manual_seed(42)

def train(model: torch.nn.Module,
          num_epochs: int,
          train_dataloader: torch.utils.data.DataLoader,
          valid_dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          device: torch.device):
    epochs=num_epochs
    loss_hist_train = [0]*epochs
    accuracy_hist_train = [0]*epochs
    loss_hist_valid = [0]*epochs
    accuracy_hist_valid=[0]*epochs
    model = model.to(device)
    train_time_start = timer()
    for epoch in tqdm(range(epochs)):
        model.train()
        for x_batch, y_batch in train_dataloader:
            #x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            #x_batch, y_batch = x_batch.unsqueeze(1), y_batch.unsqueeze(1)
            
            pred = model(x_batch)[:,0]
            #print('pred-->',pred)
            #print('y--->',y_batch)
            #print('is correct--->',((pred>=0.5).float() == y_batch).float())
            loss=loss_fn(pred,y_batch.float())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss_hist_train[epoch]+=loss.item()*y_batch.size(0)
            is_correct=((pred>=0.5).float() == y_batch).float()
            #print(is_correct.sum().item())
            accuracy_hist_train[epoch]+=is_correct.sum().item()
        loss_hist_train[epoch]/=len(train_dataloader.dataset)
        accuracy_hist_train[epoch]/=len(train_dataloader.dataset)
        model.eval()
        with torch.no_grad():
            for x_batch, y_batch in valid_dataloader:
                #x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                #x_batch, y_batch = x_batch.unsqueeze(1), y_batch.unsqueeze(1)
                pred = model(x_batch)[:,0]
                #print('pred BEFORE-->',pred)
                #pred = (torch.sigmoid(pred) + 0.5)
                #print('pred-->',pred)
                #print('y--->',y_batch)
                #print('is correct--->',((pred>=0.5).float() == y_batch).float())
                loss=loss_fn(pred,y_batch.float())
                loss_hist_valid[epoch]+=loss.item()*y_batch.size(0)
                is_correct=((pred>=0.5).float() == y_batch).float()
                accuracy_hist_valid[epoch]+=is_correct.sum().item()
                #print('ACC',accuracy_hist_valid[epoch])
        loss_hist_valid[epoch]/=len(valid_dataloader.dataset)
        accuracy_hist_valid[epoch]/=len(valid_dataloader.dataset)
        #print('ACC_TOTAL---->',accuracy_hist_valid[epoch])
        if (epoch+1) % 5 == 0:
            print(f'Epoch:{epoch+1}---Train Acc:{accuracy_hist_train[epoch]*100:.3f}%---Valid Acc:{accuracy_hist_valid[epoch]*100:.3f}%')
    train_time_end = timer()
    total_train_time = print_train_time(start=train_time_start, 
                                           end=train_time_end,
                                           device=str(next(model.parameters()).device))
    return [loss_hist_train, loss_hist_valid, accuracy_hist_train,accuracy_hist_valid], total_train_time

def test(model: torch.nn.Module,
          test_dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          device: torch.device):
    loss_test = 0.0
    accuracy_test=0.0
    model = model.to(device)
    with torch.no_grad():
        for x_batch, y_batch in test_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            pred = model(x_batch)[:,0]
            loss=loss_fn(pred,y_batch.float())
            #print(y_batch.size(0))
            loss_test+=loss.item()*y_batch.size(0)
            pred_round = (pred>=0.5).float()
            pred_log = torch.sigmoid(pred)
            #print(pred_log)
            is_correct=(pred_round == y_batch).float()
            accuracy_test+=is_correct.sum().item()
            #print('ACC',accuracy_hist_valid[epoch])
    loss_test/=len(test_dataloader.dataset)
    accuracy_test/=len(test_dataloader.dataset)
    confmat = confusion_matrix(y_true=y_batch.cpu(), y_pred=pred_round.cpu())
    precision = precision_score(y_true=y_batch.cpu(), y_pred=pred_round.cpu())
    recall = recall_score(y_true=y_batch.cpu(), y_pred=pred_round.cpu())
    f1 = f1_score(y_true=y_batch.cpu(), y_pred=pred_round.cpu())
    fpr, tpr, thresholds = roc_curve(y_true=y_batch.cpu(), y_score=pred_log.cpu())
    roc_auc = roc_auc_score(y_true=y_batch.cpu(), y_score=pred_log.cpu())
    #print(f'fpr:{fpr}, tpr:{tpr}')
    fnr = 1 - tpr
    eer_1 = fpr[np.nanargmin(np.abs(fnr - fpr))]
    eer_2 = fnr[np.nanargmin(np.abs(fnr - fpr))]
    eer = (eer_1+eer_2)/2
    print(f'---Test Acc:{accuracy_test*100:.3f}%')
    return [loss_test,confmat,precision,recall,f1,roc_auc,eer, accuracy_test]
    
    

In [ ]:

#neural network model class
class CNN(nn.Module):
    def __init__(self, max_pool_sizes, in_features = 1, out_dim=1, conv_out = 3840):
        super().__init__()
        self.conv1 = nn.Conv2d(in_features, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(max_pool_sizes[0], max_pool_sizes[0])
        self.fc1 = nn.Linear(conv_out, 128)
        self.fc2 = nn.Linear(128, out_dim)
        self.relu=nn.ReLU()
        self.dropout=nn.Dropout(p=0.25)

    def forward(self, x: torch.Tensor):
        x = x.unsqueeze(1)
        
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))
        x = self.dropout(x)
        x = flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        
class RNN(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_size, dropout=0.0, num_layers=1, bidirectional=False, num_classes=1, type='rnn'):
        super().__init__()
        types_rnn = {'rnn':nn.RNN(out_channels, hidden_size, dropout=dropout, num_layers=num_layers, bidirectional=bidirectional, batch_first=True),
                   'lstm':nn.LSTM(out_channels, hidden_size, dropout=dropout, num_layers=num_layers, bidirectional=bidirectional, batch_first=True),
                   'gru':nn.GRU(out_channels, hidden_size, dropout=dropout, num_layers=num_layers, bidirectional=bidirectional, batch_first=True),}
        self.rnn = types_rnn[type]
        hidden_size = hidden_size * 2 if bidirectional else hidden_size
        self.fc = nn.Linear(hidden_size, num_classes)
        self.relu=nn.ReLU()
        #self.softmax=nn.Softmax()

    def forward(self, x):
        x = flatten(x,1)
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x

class Conv2dRNN(nn.Module):
    def __init__(self, max_pool_sizes, in_channels = 1, out_channels=1, hidden_size=512, dropout=0.0, num_layers=1, bidirectional=False, num_classes=1, type='rnn'):
        super().__init__()
        types_rnn = {'rnn':nn.RNN(out_channels, hidden_size, dropout=dropout, num_layers=num_layers, bidirectional=bidirectional, batch_first=True),
                   'lstm':nn.LSTM(out_channels, hidden_size, dropout=dropout, num_layers=num_layers, bidirectional=bidirectional, batch_first=True),
                   'gru':nn.GRU(out_channels, hidden_size, dropout=dropout, num_layers=num_layers, bidirectional=bidirectional, batch_first=True),}
        self.rnn = types_rnn[type]
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(max_pool_sizes[0], max_pool_sizes[0])
        #self.rnn = nn.RNN(out_channels, hidden_size, dropout=dropout, num_layers=num_layers, bidirectional=bidirectional, batch_first=True)
        hidden_size = hidden_size * 2 if bidirectional else hidden_size
        self.fc = nn.Linear(hidden_size, num_classes)
        self.relu=nn.ReLU()
        self.dropout=nn.Dropout(p=0.25)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))
        x = self.dropout(x)
        x = flatten(x, 1)
        x = x.unsqueeze(1)
        
        x, _ = self.rnn(x)
        # x shape: (batch_size, seq_length, hidden_size)

        x = x[:, -1, :]  # Take the last output of LSTM 
        x = self.fc(x)
        # x shape: (batch_size, num_classes)

        return x

In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split

class sub_dt(Dataset):
    def __init__(self, X, y,   transform=None):
        assert len(X)==len(y), 'Different Length'
        self.x = X
        self.y = y
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        sample = self.transform(self.x[idx]), self.y[idx]
        return sample
n_fft = 2048
win_length = None
hop_length = 512
n_mels = 256
n_mfcc = 256
mfcc_transform = T.MFCC(
    sample_rate=16000,
    n_mfcc=n_mfcc,
    melkwargs={"n_fft": n_fft,"n_mels": n_mels,"hop_length": hop_length,"mel_scale": "htk",}
)
mfcc_transform = mfcc_transform.to(device)

#generator1 = torch.Generator().manual_seed(42)
#alpha = int(np.floor(.1*len(dt_training)))

dt_training = sub_dt(X_training, y_training, transform = mfcc_transform)
#dt_testing, dt_training = random_split(dt_training, [alpha, len(dt_training)-alpha], generator=generator1)
dt_testing = sub_dt(X_testing, y_testing, transform = mfcc_transform)
dt_valid = sub_dt(X_valid, y_valid, transform = mfcc_transform)
batch_size = 8
train_dataloader = DataLoader(dt_training, batch_size=batch_size, shuffle=True, drop_last=True)
valid_dataloader = DataLoader(dt_valid, batch_size=batch_size,shuffle=False, drop_last=True)
test_dataloader = DataLoader(dt_testing, batch_size=len(dt_testing), shuffle=False, drop_last=False)
#test_dataloader = DataLoader(dt_testing, batch_size=batch_size, shuffle=True, drop_last=True)
print(f'Length of train dataloader: {len(train_dataloader)} batches of:{batch_size}')
print(f'Length of valid dataloader: {len(valid_dataloader)} batches of:{batch_size}')
print(f'Length of testing dataloader: {len(test_dataloader)} batches of:{len(dt_testing)}')

In [ ]:
from numpy import floor

output_rnn = dt_testing[0][0].shape[0]*dt_testing[0][0].shape[1]
print('RNN: ',output_rnn)

def cnn_output_shape(dim_in, kernel_size, stride):
    h_out = int(floor((dim_in[0] - (kernel_size[0]-1) - 1 + stride[0]) / stride[0]))
    w_out = int(floor((dim_in[1] - (kernel_size[1]-1) - 1 + stride[1]) / stride[1]))
    return h_out,w_out

kernel_size = (2,2)
stride=(2,2)
conv1 = cnn_output_shape(dim_in=(n_mfcc,63), kernel_size=kernel_size, stride=stride)
print(conv1)
conv2 = cnn_output_shape(dim_in=conv1, kernel_size=kernel_size, stride=stride)
print(conv2)
conv3 = cnn_output_shape(dim_in=conv2, kernel_size=kernel_size, stride=stride)
print(conv3)
conv4 = cnn_output_shape(dim_in=conv3, kernel_size=kernel_size, stride=stride)
print(conv4)
conv_out = 256 * conv4[0] * conv4[1]
print('Conv: ',conv_out)

In [ ]:
model = CNN(max_pool_sizes=(kernel_size, stride), in_features=1, out_dim=1, conv_out = conv_out)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01)
#print(model)
hist,time = train(model=model, 
             num_epochs=3, 
             train_dataloader=train_dataloader, 
             valid_dataloader=valid_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             device=device
            )
plot_loss_acc(hist)
testing = test(model=model,
              test_dataloader=test_dataloader,
              loss_fn=loss_fn,
              device=device)
print(f'Testing Loss:{testing[0]:.1f}---Testing Acc:{testing[-1]*100:.2f}%')
print(f'matrix:{testing[1]}, precision:{testing[2]}, recall:{testing[3]}, f1:{testing[4]}, auc:{testing[5]}, eer:{testing[6]}')
#[loss_test,confmat,precision,recall,f1,roc_auc,eer, accuracy_test]

In [ ]:
def get_optim(model=None,params = None, lr=0.01, momentum = 0, type = 'SGD'):
    loss_fn = nn.BCEWithLogitsLoss()
    model.load_state_dict(params)
    return model, loss_fn, torch.optim.SGD(params=model.parameters(), lr=lr, momentum=momentum)

models_dict = {
'cnn': CNN(max_pool_sizes=(kernel_size, stride), in_features=1, out_dim=1, conv_out = conv_out),
'rnn': RNN(in_channels=1, out_channels=output_rnn, hidden_size=256, dropout=0.0, num_layers=1, bidirectional=False, num_classes=1, type='rnn'),
'cnn_rnn': Conv2dRNN(max_pool_sizes=(kernel_size, stride), in_channels=1, out_channels=conv_out, hidden_size=256,dropout=0.0, num_layers=1, bidirectional=False, num_classes=1, type='rnn'),
'lstm': RNN(in_channels=1, out_channels=output_rnn, hidden_size=256, dropout=0.0, num_layers=1, bidirectional=False, num_classes=1, type='lstm'),
'cnn_lstm': Conv2dRNN(max_pool_sizes=(kernel_size, stride), in_channels=1, out_channels=conv_out, hidden_size=256,dropout=0.0, num_layers=1, bidirectional=False, num_classes=1, type='lstm'),
'gru': RNN(in_channels=1, out_channels=output_rnn, hidden_size=256, dropout=0.0, num_layers=1, bidirectional=False, num_classes=1, type='gru'),
'cnn_gru': Conv2dRNN(max_pool_sizes=(kernel_size, stride), in_channels=1, out_channels=conv_out, hidden_size=256,dropout=0.0, num_layers=1, bidirectional=False, num_classes=1, type='gru')
}

models = ['cnn','rnn','cnn_rnn','lstm','cnn_lstm','gru','cnn_gru']#'cnn','rnn','cnn_rnn','lstm','cnn_lstm','gru','cnn_gru'

learning_rates = [0.01,0.001]#0.01,
momentums = [0,0.1]#,0.1
num_epochs=15
trials=10
results = {}
for model_iter in models:
    params = models_dict[model_iter].state_dict()
    for lr in learning_rates:
        for momento in momentums:
            for iter in range(trials):
                experiment = model_iter+ '/'+str(iter) + '/' + str(lr)+'/'+str(momento)
                model, loss_fn, optimizer = get_optim(models_dict[model_iter],params, lr, momento, 'SGD')
                print('\n\n'+'+'*5, experiment,'+'*5)
                training, time = train(model=model, 
                     num_epochs=num_epochs, 
                     train_dataloader=train_dataloader, 
                     valid_dataloader=valid_dataloader,
                     loss_fn=loss_fn,
                     optimizer=optimizer,
                     device=device
                    )
                testing = test(model=model,
                      test_dataloader=test_dataloader,
                      loss_fn=loss_fn,
                      device=device)
                results[experiment] = {'model':model_iter, 
                                       'lr':str(lr), 
                                       'momentum': str(momento), 
                                       'training_hist':training, 
                                       'testing_hist':testing, 
                                       'time':time, 
                                       'training_acc':training[2][-1],
                                       'valid_acc': training[3][-1],
                                       'test_acc': testing[-1],
                                       'test_confmatrix': testing[1],
                                       'test_precision': testing[2],
                                       'test_recall': testing[3],
                                       'test_f1': testing[4],
                                       'test_auc': testing[5],
                                       'test_eer': testing[6],
                                      }

#results
        
        

In [ ]:
for _ in results.keys():
    print('\n{:=^50}'.format(_))
    print('Testing Loss:{:.3f}---Testing Accuracy:{:.3f}%'.format(results[_]['testing_hist'][0],results[_]['testing_hist'][-1]*100))
    print('Training Time: {:.3f} sec.'.format(results[_]['time']))
    plot_loss_acc(results[_]['training_hist'])
    

In [ ]:
import pickle 
data = 'results_256_10iters_metrics_0_5.pkl'
with open(data, 'wb') as f:
    pickle.dump(results, f)
        
with open(data, 'rb') as f:
    loaded_dict = pickle.load(f)
    

In [ ]:
results

In [ ]:
dict_agg={'time':'mean',
         'training_acc':'mean',
         'valid_acc':'mean',
         'test_acc':'mean',
        'test_precision':'mean',
        'test_recall':'mean',
        'test_f1':'mean',
        'test_auc':'mean',
         'test_eer':'mean'
        }
import pandas as pd
df = pd.DataFrame.from_dict(results, orient='index')
df.drop(['training_hist', 'testing_hist','test_confmatrix'], axis=1,inplace=True)
df.reset_index(inplace = True, drop=True)
df_all = df.groupby(['model','lr', 'momentum']).agg(dict_agg).reset_index()

df_all

In [ ]:
df_model = df.groupby(['model']).agg(dict_agg).reset_index()

df_model.sort_values('test_acc')

In [ ]:
df_model_lr = df.groupby(['model','lr']).agg(dict_agg).reset_index()

df_model_lr

In [ ]:
df

In [ ]:
import seaborn as sns
figure, axes = plt.subplots(2,2,sharex=False, figsize=(10,8), layout='tight')
y_list=['time', 'training_acc', 'valid_acc', 'test_acc']
for i, ax in enumerate(axes.flat):
    
    sns.boxplot(df, x='model',y=y_list[i], ax=ax)
    ax.set_title(y_list[i])
    '''
sns.boxplot(df, x='model',y='time', ax=axes[0][0])
sns.boxplot(df, x='model',y='training_acc', ax=axes[0][1])
sns.boxplot(df, x='model',y='valid_acc', ax=axes[1][0])
sns.boxplot(df, x='model',y='test_acc', ax=axes[1][1])'''

In [ ]:
figure, axes = plt.subplots(2,2,sharex=False, figsize=(10,8), layout='tight')
y_list=['time', 'training_acc', 'valid_acc', 'test_acc']
for i, ax in enumerate(axes.flat):
    
    sns.boxplot(df[df['model'].isin(['cnn', 'cnn_lstm','cnn_gru'])], x='model',y=y_list[i], ax=ax)
    ax.set_title(y_list[i])

plt.show()

In [ ]:
figure, axes = plt.subplots(2,2,sharex=False, figsize=(10,8), layout='tight')
y_list=['time', 'training_acc', 'valid_acc', 'test_acc']
for i, ax in enumerate(axes.flat):
    
    sns.boxplot(df[(df['model'].isin(['cnn', 'cnn_lstm','cnn_gru'])) & (df['lr']=='0.001')], x='model',y=y_list[i], ax=ax)
    ax.set_title(y_list[i])

plt.show()